DSSM Training

In [8]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [22]:
query = pd.read_csv("~/data/queries_train_indices.csv", header = None)
display(query)
query.isnull().sum()

,0,1
0,121352,1924 2363
1,634306,29570 321 193 198
2,920825,131 8242 1880 36968
3,510633,4711 38 31
4,737889,39755 99
...,...,...
808726,633855,752 703 2420 321
808727,1059728,6398 36572 136
808728,210839,1087 22
808729,908165,4 685 45818 233


0        0
1    15181
dtype: int64

In [2]:
def sparse_to_dense(idx, vocab_len):
    index_tensor = torch.LongTensor([idx])
    value_tensor = torch.Tensor([1]*len(idx))
    dense_tensor = torch.sparse.FloatTensor(index_tensor, value_tensor, torch.Size([vocab_len,])).to_dense()
    return dense_tensor

In [3]:
sparse_to_dense([1,3,5,3],6)

tensor([0., 1., 0., 2., 0., 1.])

In [4]:
indices = [1,3,5,3]
length = 6
i = torch.LongTensor([indices])
v = torch.Tensor([1]*len(indices))
torch.sparse.FloatTensor(i, v, torch.Size([length,])).to_dense()

tensor([0., 1., 0., 2., 0., 1.])

In [5]:
torch.sparse.FloatTensor(3)


tensor(indices=tensor([], size=(1, 0)),
       values=tensor([], size=(0,)),
       size=(3,), nnz=0, layout=torch.sparse_coo)

In [6]:
torch.sparse.FloatTensor(3)


tensor(indices=tensor([], size=(1, 0)),
       values=tensor([], size=(0,)),
       size=(3,), nnz=0, layout=torch.sparse_coo)